# TP 2 : Régression logistique

## 1 Régularisation de Tikhonov

### 1.1 : Étude de la fonction objectif :

**Calcul du gradient :**

$\frac{df_1}{dw_0} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i}{1+exp(y_i(x_i^T*w + w_0))}}$


$\frac{df_1}{dw} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i*x_i}{1+exp(y_i(x_i^T*w + w_0))}} + \rho * w$

**Calcul de la hessienne :**

$H = \frac 1 n * \sum_i{\frac{y_i^2}{(2cosh(y_i/2(x_i^T*w + w_0))^2}} * \begin{bmatrix}1 \\ x_i \end{bmatrix} \begin{bmatrix}1 \\ x_i \end{bmatrix}^T + \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix}  $

Montrons que $H$ est définie positive. Posons $a_i = \frac{y_i}{\sqrt n (2cosh(y_i/2(x_i^T*w + w_0))}$ et $v_i = \begin{bmatrix}1 \\ x_i \end{bmatrix}$, on a :

$\forall u \in \mathbb R^{p+1}, u^T H u = u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u + \sum_i{(u^T a_i v_i) (v_i^T a_i u) }$

* $u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u = \begin{bmatrix} 0 & {\rho \sum_{i=2}^{p+1}{u_i}} & {...} & {\rho \sum_{i=2}^{p+1}{u_i}} \end{bmatrix} \begin{bmatrix} u_1 \\ ... \\ u_ {p+1} \end{bmatrix} = \rho (\sum_{i=2}^{p+1}{u_i})^2 > 0$
* $\sum_i{(u^T a_i v_i) (v_i^T a_i u) } = \sum_i{(u^T a_i v_i)^2 } > 0$

Ainsi $ u^T H u > 0$. 

Et donc $f_1$ est convexe

### 1.2 : Programmation :

In [16]:
import numpy as np
from scipy.optimize import check_grad

def objective(w, X, Y, rho, verbose=False):
    """ Renvoie les données numériques du problème
    :param w: Le vecteur [w0 w] de l'énoncé. On attend une liste de longueur p+1.
    :param X: La matrice des vecteurs lignes xi. On attend une matrice de dimension n lignes * p+1 colonnes.
        La 1ère colonne est composée uniquement de 1.
    :param Y: Le vecteur des yi. Liste de longueur n.
    :param rho: Le paramètre de régularisation.
    
    :return: La valeur de la fonction, son gradient et sa hessienne."""
    assert (len(Y), len(w)) == X.shape, "Erreur de dimensions"
    n = len(Y)
    p = len(w) - 1
    # Valeur de la fonction au point w :
    val = rho/2 * np.linalg.norm(w, 2)**2
    for i in range(n):
        s = np.exp(-Y[i]*np.dot(X[i,:], w))
        if verbose:
            print("Étape %d, Val = %f" % (i, val))
            print("s:", s)
        val += 1/n*np.log(1 + s)
    
    # Gradient de la fonction au point w 
    grad = np.zeros(p+1)
    grad[1:] = rho * w[1:]
    for i in range(n):
        grad += 1/n * (Y[i]*X[i])/(1 + np.exp(Y[i] * np.dot(X[i,:], w)))
        
    # Matrice hessienne :
    hess = np.identity(p+1)*rho
    hess[0,0] = 0
    for i in range(n):
        v = X[i,:][np.newaxis]
        hess += 1/n * (Y[i] / (2*np.cosh(Y[i]/(2*np.dot(X[i,:], w)))))**2 * v.T.dot(v)
    return val, grad, hess

In [17]:
from cervicalcancerutils import load_cervical_cancer

X,y = load_cervical_cancer("riskfactorscervicalcancer.csv")
(n, p) = X.shape

# On rajoute une colonne de 1 à X :
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Affichage des erreurs sur le gradient :
n_verif = 10 # Nombre de vérification à effectuer
print("Écart absolu | Écart relatif")
print("----------------------------")
for i in range(n_verif):
    rho = np.random.random()
    w = np.random.random(p+1)
    f = lambda w:objective(w, X, y, rho)[0]
    grad_f = lambda w:objective(w, X, y, rho)[1]
    err = check_grad(f, grad_f, w)
    err_rel = err/np.linalg.norm(grad_f(w))
    print("%10.3f   | %5.1f%%" % (err, err_rel))

Écart absolu | Écart relatif
----------------------------
     0.953   |   2.0%
     0.965   |   0.4%
     0.992   |   0.7%
     0.913   |   1.0%
     0.958   |   0.4%
     0.908   |   1.3%
     0.905   |   1.2%
     0.932   |   0.7%
     0.928   |   0.6%
     0.807   |   2.3%


### 1.3 : Méthode de Newton :

In [18]:
def newton(objective, start, epsilon=0.1, max_iter=1000,verbose=False):
    """ Renvoie le point qui minimise la fonction objectif.
    :param objective: une fonction qui prend un vecteur et renvoie (valeur, gradient, hessienne) en ce point.
    :param start: le point de départ
    
    :return: le point qui minimise la fonction.
    """
    
    i = 1
    
    proceed = True
    i = 1
    point = start
    while proceed:
        val, grad, hess = objective(point)
        norm_grad = np.linalg.norm(grad)
        if verbose:
            print("Itération %d, val = %f, ||grad|| = %f" % (i, val, norm_grad))
        point -= np.linalg.inv(hess).dot(grad)
        i += 1
        proceed = i <= max_iter and norm_grad < epsilon

optimum = newton(lambda w:objective(w, X, y, rho), np.zeros_like(w))

/home/loic/.local/lib/python3.5/site-packages/ipykernel/__main__.py:36: RuntimeWarning: divide by zero encountered in double_scalars


LinAlgError: Singular matrix